In [196]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from sklearn.metrics import r2_score
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.animation as animation # animation
import imageio 

import glob
import pandas as pd
import os
from shapely.geometry import shape
import csv
#from mapxy import mapxy
import pyproj
#from ps_latlon_conversion import xyscale_south
import pyproj
#import ogr
from osgeo import gdal, osr
import matplotlib.pyplot as plt
import fiona
import rasterio
import rasterio.transform
import rasterio.mask
from fiona import Feature, Geometry
from shapely.geometry import mapping, shape

In [228]:
#extracting the datasets and creating two new dataframes, which contais the common years

#Calving flux
data_dir_ice_calving = '/Users/francesco/Desktop/Thesis/exploration_ice_calving/'
calving_df= pd.read_csv( data_dir_ice_calving+ 'Glacier_ice_calving_withice_shelves_flux_greene.csv', index_col = 0)
calving_df = calving_df.sort_index()
index_calving = calving_df.index.values
calving_columns= calving_df.columns


#PER ORA NON SERVE
#Basal melt. Does it make sense? the values are not even number, just raserio objects
data_dir_basal_melt = '/Users/francesco/Desktop/Thesis/exploration_basal_melting/'
melt_df = pd.read_csv(data_dir_basal_melt + 'basal_melt_df.csv', index_col = 0)
melt_df = melt_df.sort_index()
index_melt = melt_df.index.values
melting_columns = melt_df.columns



#Creating the new dataframes
common_columns_melt_calv = np.intersect1d(calving_columns, melting_columns)

melt_df_common = melt_df[common_columns_melt_calv]
calving_df_common = calving_df[common_columns_melt_calv]

In [229]:
datadir ='/Users/francesco/Desktop/Thesis/Data/'

df_melt_rates = pd.read_csv(datadir + 'Merged_Integrated_melt_rates.csv')
df_err_melt_rates = pd.read_csv(datadir + 'Merged_Integrated_melt_err_rates_second_method.csv')

In [230]:
#Check if index_melt and index_calving are the same

if np.array_equal(index_melt, index_calving):
    print('The two dataframes have the same index')
else:
    print('The two dataframes have different index')

The two dataframes have the same index


In [231]:
#Extracting the info from the shapefile

shapefile_path = '/Users/francesco/Desktop//Thesis/Data/ice_shelf.shp'

ids = []
Names = []
lat_lon_coords = []
regions = []
areas = []

shapefile=fiona.open(shapefile_path)

print(shapefile.schema['properties'])

for feature in shapefile:

    id = feature['properties']['id']
    name=feature['properties']['name']
    geometry=feature['geometry']
    if geometry is None:
        continue
    region = feature['properties']['regions']
    lat = feature['properties']['latitude']
    lon = feature['properties']['longitude']
    area = feature['properties']['area']

    bounds=shape(geometry).bounds

    #Taking the info
    ids.append(id)
    Names.append(name)
    lat_lon_coords.append([lat,lon])
    regions.append(region)
    areas.append(area)




{'id': 'int:10', 'name': 'str:30', 'regions': 'str:11', 'type': 'str:2', 'is_index': 'int:10', 'ice_shelf_': 'str:50', 'measures_n': 'str:50', 'latitude': 'float:24.15', 'longitude': 'float:24.15', 'thickness_': 'float:24.15', 'thicknes_1': 'float:24.15', 'smb_mean_9': 'float:24.15', 'smb_mean_1': 'float:24.15', 'ss_gl_flux': 'float:24.15', 'ss_gl_fl_1': 'float:24.15', 'ss_calving': 'float:24.15', 'ss_calvi_1': 'float:24.15', 'model_gl_f': 'float:24.15', 'instant_co': 'float:24.15', 'mass_loss_': 'float:24.15', 'mass_los_1': 'float:24.15', 'mass_los_2': 'float:24.15', 'mass_los_3': 'float:24.15', 'mass_los_4': 'float:24.15', 'mass_los_5': 'float:24.15', 'mass_los_6': 'float:24.15', 'mass_los_7': 'float:24.15', 'mass_los_8': 'float:24.15', 'mass_los_9': 'float:24.15', 'mass_los10': 'float:24.15', 'mass_los11': 'float:24.15', 'mass_los12': 'float:24.15', 'mass_los13': 'float:24.15', 'mass_los14': 'float:24.15', 'mass_los15': 'float:24.15', 'mass_los16': 'float:24.15', 'mass_los17': 'floa

In [232]:
#create a new dataframe with the info
df = pd.DataFrame({'name':Names,'lat_lon_coords':lat_lon_coords,'regions':regions, 'area': areas}, index=ids)
df = df.sort_index()

In [190]:
calving_2007 = calving_df_common['2007']
calving_2008 = calving_df_common['2008']

melt_2007 = melt_df_common['2007']
melt_2008 = melt_df_common['2008']

In [233]:
df_2007 = pd.DataFrame({'name':Names, 'area':areas, 'calving':calving_2007}, index=ids)

In [166]:
df_2007
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#df_2007


In [244]:
#Importing the Rignot dataset
df_rignot = pd.read_excel('table.xlsx')
df_rignot = df_rignot.set_index('id')
df_rignot = df_rignot.sort_index()

#NB: THE RIGNOT INDEX SHOULD HAVE THE SAME VALUES OF THE DF INDEX, BEACUSE I WROTE 
#IT MANUALLY ON THE TABLE EXCEL FILE
df_rignot_index = df_rignot.index.values
df_rignot_index

#df_rignot

/Users/francesco/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


array([  6.,   7.,   8.,  10.,  12.,  16.,  17.,  24.,  31.,  33.,  34.,
        35.,  36.,  37.,  38.,  39.,  45.,  46.,  50.,  51.,  52.,  53.,
        54.,  55.,  56.,  57.,  58.,  60.,  63.,  64.,  65.,  66.,  68.,
        70.,  71.,  73.,  80.,  83.,  86.,  90.,  91.,  98.,  99., 100.,
       102., 104., 110., 114., 115., 118., 119., 120., 121., 122., 123.,
       124., 125., 155., 156., 157., 158., 159., 161., 162.,  nan])

In [245]:
df_rignot

,name,Lat,Lon,Survey Area m2,Ice-front flux,Ice-front flux error Gt/yr,Basal melting tot Gt/yr,Basal Melt tot error Gt/yr,Basal melting per meter m/yr,Basal Melting per meter Error m/yr,Actual Area m2
id,,,,,,,,,,,
6.0,LarsenD,-70.7032,-61.72400,2.157687e+10,6.3,1.0,1.4,14.0,0.06,0.6,2.254808e+10
7.0,Larsen_F,-74.0720,-61.23160,7.599880e+08,0.6,1.0,1.9,0.6,2.30,0.8,8.278330e+08
8.0,LarsenC,-67.4601,-62.72942,4.562956e+10,31.3,3.0,20.7,67.0,0.45,1.0,4.646549e+10
10.0,Shirase,-69.9511,38.54770,5.098000e+08,9.6,1.0,5.7,1.0,6.98,2.0,8.206200e+08
12.0,Rayner_Thyer,-67.7276,48.45490,5.876990e+08,7.8,0.6,6.7,1.0,10.46,2.0,6.407370e+08
16.0,WilmaRobertDowner,-67.0348,56.59810,8.215400e+08,0.8,0.4,10.0,0.6,11.72,0.7,8.575300e+08
17.0,Edward_VIII,-66.7300,56.43000,3.943000e+08,0.3,0.1,4.2,0.8,10.22,2.0,4.113000e+08
24.0,Tracy_Tremenchus,-65.8641,101.21200,2.723070e+09,0.2,0.1,2.7,2.0,1.49,0.7,2.844700e+09
31.0,Lillie,-70.7889,163.99700,7.215110e+08,0.5,0.1,3.4,0.3,4.40,0.4,7.697110e+08


In [246]:
# i want to select the ids which are in common 
common_index = np.intersect1d(index_melt, df_rignot_index)
common_index

common_names = df.loc[common_index]['name']
common_names


6                LarsenD
7                LarsenF
8                LarsenC
10               Shirase
12          Rayner_Thyer
16     WilmaRobertDowner
17           Edward_VIII
24      Tracy_Tremenchus
31                Lillie
33               LarsenG
34                 Ronne
35              Filchner
36        Brunt_Stancomb
37         Riiser-Larsen
38                  Quar
39               Ekstrom
45             George_VI
46                  Bach
50               Wilkins
51               LarsenB
52                Stange
53               Venable
54              Cosgrove
55           Pine_Island
56              Thwaites
57               Crosson
58                Dotson
60                  Getz
63                  Land
64             Nickerson
65            Sulzberger
66             Swinburne
68               Withrow
70             Ross_West
71             Ross_East
73             Drygalski
80               Aviator
83               Mariner
86               Rennick
90                  Cook


In [252]:

#Selecting the melts rates of the glaciers which are in common
#N.B: the index now are not the same! they are still the automatic ones
df_melt_rates_common = df_melt_rates[df_melt_rates['Names'].isin(common_names)]

In [253]:
#Now I want to extract the index of the dataframe df_rignot 
#which are the ones for which the name between df_rignot and 
#df_melt_rates_common are the same

df_rignot_common = df_rignot.index[df_rignot['name'].isin(common_names)]

In [254]:
df_rignot_common

Float64Index([  6.0,   8.0,  10.0,  12.0,  16.0,  17.0,  24.0,  31.0,  33.0,
               34.0,  35.0,  36.0,  37.0,  38.0,  39.0,  45.0,  46.0,  50.0,
               51.0,  52.0,  53.0,  54.0,  55.0,  56.0,  57.0,  58.0,  60.0,
               63.0,  64.0,  65.0,  66.0,  68.0,  70.0,  71.0,  73.0,  80.0,
               83.0,  86.0,  90.0,  91.0,  98.0,  99.0, 100.0, 102.0, 104.0,
              110.0, 114.0, 115.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0,
              125.0, 155.0, 156.0, 157.0, 158.0, 161.0, 162.0],
             dtype='float64', name='id')

In [255]:
df_melt_rates_common['id'] = df_rignot_common
#df_melt_rates_common = df_melt_rates_common.set_index('id')
#df_melt_rates_common


ValueError: Length of values (61) does not match length of index (64)

In [209]:
common_names.index

Int64Index([  6,   7,   8,  10,  12,  16,  17,  24,  31,  33,  34,  35,  36,
             37,  38,  39,  45,  46,  50,  51,  52,  53,  54,  55,  56,  57,
             58,  60,  63,  64,  65,  66,  68,  70,  71,  73,  80,  83,  86,
             90,  91,  98,  99, 100, 102, 104, 110, 114, 115, 118, 119, 120,
            121, 122, 123, 124, 125, 155, 156, 157, 158, 159, 161, 162],
           dtype='int64')

In [174]:
calving_2007_common = calving_2007[common_index]
melting_2007_common = melt_df_common['2007'][common_index]

In [186]:
comparison_calving = pd.DataFrame({'name':common_names,'calving_me':calving_2007_common, 'calving_rignot':df_rignot['Ice-front flux'], 'percentage': df_rignot['Ice-front flux']/calving_2007_common }, index=common_index)

In [188]:
comparison_calving

,name,calving_me,calving_rignot,percentage
6.0,LarsenD,34.170000,6.3,0.184372
7.0,LarsenF,6.010000,0.6,0.099834
8.0,LarsenC,165.784390,31.3,0.188799
10.0,Shirase,23.708906,9.6,0.404911
12.0,Rayner_Thyer,29.020000,7.8,0.268780
16.0,WilmaRobertDowner,9.288108,0.8,0.086132
17.0,Edward_VIII,10.680000,0.3,0.028090
24.0,Tracy_Tremenchus,5.326545,0.2,0.037548
31.0,Lillie,2.560000,0.5,0.195312
33.0,LarsenG,6.690000,0.7,0.104634
